# Landscape indicator computation notebook

Scripts for preprocessing and generating landscape indicators using the 50km sLUC methodology
 - Deforesation, 
 - Deforestation carbon, and 
 - Cropland expansion in natural lands

Needs to be run at least twice to cache intermediate calculations if starting from a clean
working directory.

## Methodology

See doc: https://docs.google.com/document/d/1s36r6jSCGkmgQAf4Rg4X36oYYxeDnT0jvvO5yPxnDE4/edit


In [1]:
# Imports

import os

import ee
import eeUtil
import geemap.foliumap as gmap

# Initialize Earth Engine
PROJECT = os.getenv("GEE_PROJECT")
GEE_JSON = os.getenv("GEE_JSON")

assert PROJECT is not None, "Please set GEE_PROJECT environment variable"
assert (
    GEE_JSON is not None
), "Please set GEE_JSON environment variable with service account credentials"

eeUtil.init()

In [2]:
# Constants / options for the landscape indicators analysis
PROJECTION = "EPSG:4326"
WORKING_FOLDER = "projects/ee-fgassert/assets/landscape_indicators_20230811"
SRC_FOLDER = "projects/ee-fgassert/assets/landscape_indicators_source"
WORLD_GEOM = ee.Geometry.Polygon(
    coords=[[[-180, -85], [-180, 85], [180, 85], [180, -85], [-180, -85]]],
    proj=PROJECTION,
    geodesic=False,
)

TARGET_YEAR = 2022  # the target year for analysis
START_YEAR = TARGET_YEAR - 20  # the start year for deforestation analysis

# scales in meters
ANALYSIS_SCALE = 100  # the working scale for overlays in the main analysis
KERNEL_SCALE = 1000  # the scale for the kernel analysis
KERNEL_RADIUS = 50000

#### ASSETS ########

####################
# These need to be created/uploaded to the source folder if they don't already exist
# generated by preprocess_esri_cropland function
ESRI_CROPLAND_2020_ASSET = f"{SRC_FOLDER}/esri_cropland_lag3_30m/2020"
# generated by preprocess_esri_cropland function
ESRI_CROPLAND_ENDYEAR_ASSET = f"{SRC_FOLDER}/esri_cropland_lag3_30m/{TARGET_YEAR}"
# derived from RESOLVE ecoregions
NON_SUBTROPIC_ASSET = f"{SRC_FOLDER}/not_subtropic"
# Noon et al. 2022 vulnerable carbon total for 2010
CARBON_ASSET = f"{SRC_FOLDER}/Vulnerable_C_Total_2010"
# ESA CCI land cover for 2010
ESA_CCI_ASSET = f"{SRC_FOLDER}/ESACCI_LC_300m_P1Y_2010_v207"

####################
# 3rd party hosted assets
HANSEN_ASSET = "UMD/hansen/global_forest_change_2022_v1_10"
FOREST_DYNAMICS_ASSET = "projects/glad/GLCLU2020/Forest_type"
INTACT_FORESTS_ASSET = "users/potapovpeter/IFL_2000"
PRIMARY_TROPICAL_ASSET = "UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1/2001"

ESRI_LULC_IC = "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
CROPLAND_2019_IC = "users/potapovpeter/Global_cropland_2019"
SBTN_ASSET = "projects/wri-datalab/SBTN/natLands_beta/naturalLands_allClasses_20230516"

FIRECCI_IC = "ESA/CCI/FireCCI/5_1"
MODIS_BA_IC = "MODIS/006/MCD64A1"

In [3]:
# create working folder if not exists
eeUtil.createFolder(WORKING_FOLDER)

In [4]:
# preprocess esri cropland before anything else


def preprocess_esri_cropland(
    lag_years: int = 3, start_year: int = 2017, end_year: int = 2022, scale: int = 30
):
    """Extract cropland from ESRI landcover dataset and save to GEE

    Resamples to 30m and saves to GEE as an image collection of annual images
    such that each year represents the maximum cropland area in the previous `lag_years`.
    This will use the asset pyramids for resampling (nearest neighbor?).

    We mosaic and export ESRI data first because each tile is in a different projection,
    which prevents us from downsampling using a mean pyramiding policy.
    """
    esri_lulc = ee.ImageCollection(ESRI_LULC_IC)
    esri_crop_annual = ee.List.sequence(start_year, end_year).map(
        lambda y: (
            esri_lulc.filterDate(ee.Date.fromYMD(y, 1, 1), ee.Date.fromYMD(y, 12, 31))
            .mosaic()
            .eq(5)
        )
    )

    esri_crop_lagged = ee.List.sequence(0, esri_crop_annual.length().subtract(lag_years + 1)).map(
        lambda i: (
            ee.ImageCollection(esri_crop_annual.slice(i, ee.Number(i).add(lag_years + 1))).reduce(
                "max"
            )
        )
    )

    for i in range(end_year - start_year - lag_years + 1):
        y = start_year + lag_years + i
        ic = f"{SRC_FOLDER}/esri_cropland_lag{lag_years}_{scale}m"
        asset_id = f"{ic}/{y}"
        eeUtil.createImageCollection(ic)
        eeUtil.findOrSaveImage(
            ee.Image(esri_crop_lagged.get(i)),
            asset_id,
            region=WORLD_GEOM,
            scale=scale,
            crs=PROJECTION,
            dtype="byte",
        )


preprocess_esri_cropland()

In [5]:
def get_sbtn_layers():
    """Caches and returns relevant SBTN layers at reduced resolution using mean pyramiding.

    All Classes values:
    Natural:     2 = natural forests, 3 = natural short vegetation, 4 = natural water, 5 = mangroves
                 6 = bare, 7 = snow, 8 = wet natural forests, 9 = natural peat forests,
                 10 = wet natural short vegetation, 11 = natural peat short vegetation
    Non-Natural: 12 = cropland, 13 = built, 14 = non-natural tree cover, 15 = non-natural short
                 vegetation, 16 = non-natural water, 17 = wet non-natural forests,
                 18 = non-natural peat forests, 19 = wet non-natural short vegetation,
                 20 = non-natural peat short vegetation.
    """
    sbtn_all_classes = ee.Image(SBTN_ASSET)
    return {
        "natural_lands": sbtn_all_classes.lte(11),
        "nonnat_forest": sbtn_all_classes.eq(ee.Image([14, 17, 18])).reduce("anyNonZero"),
        "nonnatural_excl_builtwater": sbtn_all_classes.gte(12)
        .And(sbtn_all_classes.neq(13))
        .And(sbtn_all_classes.neq(16)),
        "available_area": sbtn_all_classes.eq([4, 13, 16])
        .reduce("anyNonZero")
        .Not(),  # used as mask
    }


def get_tree_loss():
    """Caches and returns Hansen tree loss at reduced resolution using mean pyramiding."""
    hanson_data = ee.Image(HANSEN_ASSET)
    loss_year = hanson_data.select(3)
    loss_portion = hanson_data.select(1)

    return {
        "tree_loss_allyrs": loss_portion,
        f"tree_loss_to{TARGET_YEAR}": loss_portion.multiply(loss_year.gte(START_YEAR - 2000)),
    }


def get_esri_cropland():
    """Gets Esri cropland at reduced resolution using mean pyramiding."""
    return {
        "esri_cropland_2020": ee.Image(ESRI_CROPLAND_2020_ASSET),
        f"esri_cropland_{TARGET_YEAR}": ee.Image(ESRI_CROPLAND_ENDYEAR_ASSET),
    }


def get_forest_dynamics():
    """Gets Potapov forest dynamics at reduced resolution using mean pyramiding."""
    forest_dynamics = ee.Image(FOREST_DYNAMICS_ASSET)
    return {"gain": forest_dynamics.eq(3), "disturbance": forest_dynamics.eq(4)}


def get_burned_area(kernel_radius=250):
    """Gets FireCII and Modis Burned Area products

    FireCCI is a bit more accurate and reported at 250m but only runs through 2020
    MODIS Burned Area is reported at 500m but is operationally near real-time

    We run both though a kernel to smooth out the edges and expand the filter region
    """
    firecci = ee.ImageCollection(FIRECCI_IC)
    modis_ba = ee.ImageCollection(MODIS_BA_IC)

    firecci = (
        firecci.filterDate(ee.Date.fromYMD(START_YEAR, 1, 1), ee.Date.fromYMD(TARGET_YEAR, 12, 31))
        .select(0)
        .mosaic()
    )
    modis_ba = (
        modis_ba.filterDate(ee.Date.fromYMD(START_YEAR, 1, 1), ee.Date.fromYMD(TARGET_YEAR, 12, 31))
        .select(0)
        .mosaic()
    )

    layers = {"firecci": firecci, "modis_ba": modis_ba, "burned_area_mask": firecci.Or(modis_ba)}

    return {
        k: image.reduceNeighborhood(
            "sum", ee.Kernel.square(kernel_radius, "meters"), None, False, "boxcar"
        ).gt(0)
        for k, image in layers.items()
    }


def get_vulnerable_carbon_filled():
    """Computes a filled version of the Noon et al. vulnerable carbon layer for 2000.

    The original layer is at 300m for the year 2010. We fill in forest areas for the year 2000
    by looking at the difference in forest cover, and filling with the mean carbon value for forests
    in a 10km radius around the forest pixel.
    """
    carbon = ee.Image(CARBON_ASSET).unmask()
    tree_cover_2000 = ee.Image(HANSEN_ASSET).select(1).unmask()
    esa_cci = ee.Image(ESA_CCI_ASSET).unmask()

    cci_tree_cover_2010 = (
        esa_cci.gte(40).And(esa_cci.lt(100)).Or(esa_cci.eq(160)).Or(esa_cci.eq(170))
    )
    tree2000_not_2010 = tree_cover_2000.And(cci_tree_cover_2010.Not())
    carbon_kernel_mean = (
        carbon.multiply(cci_tree_cover_2010)
        .selfMask()
        .reduceNeighborhood("mean", ee.Kernel.square(10000, "meters"), "mask", False, "boxcar")
    )
    carbon_filled_2000 = carbon.unmask().where(tree2000_not_2010, carbon_kernel_mean).selfMask()

    return {"carbon": carbon, "carbon_filled_2000": carbon_filled_2000}


def get_intact_forests():
    """Get intact and primary humid tropical forest layers at 100m resolution."""
    return {
        "intact_forests": ee.Image(INTACT_FORESTS_ASSET),
        "primary_tropical_forests": ee.Image(PRIMARY_TROPICAL_ASSET),
    }


def get_cropland_2019():
    """Get potapov cropland layer for 2019 at target resolution."""
    return {"cropland_2019": ee.ImageCollection(CROPLAND_2019_IC).mosaic()}


def get_all_input_layers(cache=False):
    """Get all layers used in the analysis."""
    return {
        **get_sbtn_layers(),
        **get_esri_cropland(),
        **get_tree_loss(),
        **get_intact_forests(),
        **get_vulnerable_carbon_filled(),
        **get_burned_area(),
        **get_forest_dynamics(),
        **get_cropland_2019(),
    }

In [6]:
# functions for logical operations on 0-1 floating point rasters


def and_assume_overlap(*args):
    return ee.Image([*args]).reduce("min")


def and_assume_uniform(*args):
    return ee.Image([*args]).reduce("product")


def and_assume_disjoint(*args):
    return ee.Image([*args]).reduce("sum").subtract(1).max(0)


def or_assume_overlap(*args):
    return ee.Image([*args]).reduce("max")


def or_assume_uniform(*args):
    return inverse(inverse(args).reduce("product"))


def or_assume_disjoint(*args):
    return ee.Image([*args]).reduce("sum").min(1)


def inverse(a):
    return ee.Image(a).subtract(1).multiply(-1)

In [7]:
def compute_deforest(layers=None):
    """Calculate deforestation from forest loss excluding likely types of non-deforestation loss."""
    layers = layers or get_all_input_layers()

    tree_loss = layers[f"tree_loss_to{TARGET_YEAR}"]
    non_nat_forest = layers["nonnat_forest"]
    not_primary_forest = inverse(
        or_assume_overlap(layers["intact_forests"], layers["primary_tropical_forests"])
    )
    disturbance = layers["disturbance"]
    burned_area = layers["burned_area_mask"]
    not_subtropic = ee.Image(NON_SUBTROPIC_ASSET)

    forest_ok = and_assume_overlap(non_nat_forest, not_primary_forest)
    disturbance_ok = and_assume_overlap(disturbance, not_primary_forest)
    burned_area_ok = and_assume_uniform(burned_area, not_subtropic)
    loss_ok = or_assume_overlap(forest_ok, disturbance_ok, burned_area_ok)

    deforest = tree_loss.subtract(loss_ok).max(0)

    return {**layers, "deforest": deforest}


def compute_deforest_carbon(layers=None):
    """ """
    layers = layers or compute_deforest()

    deforest = layers["deforest"]
    deforest_carbon = deforest.multiply(layers["carbon_filled_2000"])

    return {**layers, "deforest_carbon": deforest_carbon}


def compute_cropland_expansion(layers=None):
    """ """
    layers = layers or get_all_input_layers()

    cropland_end = layers[f"esri_cropland_{TARGET_YEAR}"]
    cropland_2020 = layers["esri_cropland_2020"]
    natural = layers["natural_lands"]
    cropland_expansion = cropland_end.subtract(cropland_2020).max(0)
    cropland_reduction = cropland_2020.subtract(cropland_end).max(0)
    natcrop_expansion = and_assume_disjoint(cropland_expansion, natural)
    natcrop_reduction = and_assume_disjoint(cropland_expansion, natural)

    return {
        **layers,
        "cropland_expansion": cropland_expansion,
        "cropland_reduction": cropland_reduction,
        "natcrop_expansion": natcrop_expansion,
        "natcrop_reduction": natcrop_reduction,
    }


def get_derived_layers(cache=False):
    """Compute the derived land use change layers before kernels"""
    layers = get_all_input_layers()
    layers = compute_deforest(layers)
    layers = compute_deforest_carbon(layers)
    layers = compute_cropland_expansion(layers)

    for k, image in layers.items():
        layers[k] = image.updateMask(layers["available_area"])
        if cache:
            asset_id = f"{WORKING_FOLDER}/{k}_{ANALYSIS_SCALE}m"
            image = (
                image.unmask()
                .set_default_projection(PROJECTION, None, 30)
                .reduceResolution("mean", True)
            )
            layers[k] = eeUtil.findOrSaveImage(
                image,
                asset_id,
                region=WORLD_GEOM,
                scale=ANALYSIS_SCALE,
                crs=PROJECTION,
                dtype="float",
            )

    return layers

In [8]:
def compute_kernels(layers=None):
    """computed kerneled layers"""
    layers = layers or get_derived_layers()

    mask = layers["available_area"]
    kernel_layers = {
        "tree_loss_kernel": layers[f"tree_loss_to{TARGET_YEAR}"],
        "deforest_kernel": layers["deforest"],
        "deforest_carbon_kernel": layers["deforest_carbon"],
        "natcrop_expansion_kernel": layers["natcrop_expansion"],
        "natcrop_reduction_kernel": layers["natcrop_reduction"],
        "nonnatural_kernel": layers["nonnatural_excl_builtwater"],
    }

    kernel = ee.Kernel.circle(radius=KERNEL_RADIUS, units="meters")
    for k, image in kernel_layers.items():
        image = image.updateMask(mask)
        kernel_layers[k] = image.reduceNeighborhood("mean", kernel, "mask", False)

    return kernel_layers


def compute_indicators(layers=None):
    """compute final landscape indicators"""
    layers = layers or compute_kernels()

    natural_crop_reduction_kernel = layers["natcrop_reduction_kernel"]
    natural_crop_conversion_kernel = layers["natcrop_expansion_kernel"]
    deforest_kernel = layers["deforest_kernel"]
    deforest_carbon_kernel = layers["deforest_carbon_kernel"]
    non_nat_kernel = layers["nonnatural_kernel"]
    forest_loss_kernel = layers["tree_loss_kernel"]

    # allow shifting ag
    natcrop_net_conversion = natural_crop_conversion_kernel.subtract(
        natural_crop_reduction_kernel
    ).max(0)

    # avoid divide by zero errors and cap at 1ha/ha
    tree_loss_by_human_lu = forest_loss_kernel.divide(non_nat_kernel.add(0.000001)).min(1).max(0)
    deforest_by_human_lu = deforest_kernel.divide(non_nat_kernel.add(0.000001))

    excess_deforest_per_ha_human_lu = deforest_by_human_lu.where(deforest_by_human_lu.lt(1), 1)
    deforest_by_human_lu = deforest_by_human_lu.min(1)
    deforest_carbon_by_human_lu = deforest_carbon_kernel.divide(
        non_nat_kernel.add(0.000001)
    ).divide(excess_deforest_per_ha_human_lu)

    natural_crop_conversion_by_human_lu = (
        natural_crop_conversion_kernel.divide(non_nat_kernel.add(0.000001)).min(1).max(0)
    )
    natural_crop_net_conversion_by_human_lu = (
        natcrop_net_conversion.divide(non_nat_kernel.add(0.000001)).min(1).max(0)
    )

    # normalize to annual values
    tree_loss_by_human_lu = tree_loss_by_human_lu.divide(20)
    deforest_by_human_lu = deforest_by_human_lu.divide(20)
    deforest_carbon_by_human_lu = deforest_carbon_by_human_lu.divide(20).multiply(
        3.66
    )  # convert to CO2
    natural_crop_conversion_by_human_lu = natural_crop_conversion_by_human_lu.divide(
        TARGET_YEAR - 2020
    )
    natural_crop_net_conversion_by_human_lu = natural_crop_net_conversion_by_human_lu.divide(
        TARGET_YEAR - 2020
    )

    return {
        "deforest_by_human_lu": deforest_by_human_lu,
        "deforest_carbon_by_human_lu": deforest_carbon_by_human_lu,
        "tree_loss_by_human_lu": tree_loss_by_human_lu,
        "natural_crop_conversion_by_human_lu": natural_crop_conversion_by_human_lu,
        "natural_crop_net_conversion_by_human_lu": natural_crop_net_conversion_by_human_lu,
    }


def compute_viz_layers(indicator_layers=None):
    indicator_layers = indicator_layers or compute_indicators()
    cropland = get_cropland_2019()["cropland_2019"]
    nonnat = get_sbtn_layers()["nonnatural_excl_builtwater"]

    layers = {}
    for k, image in indicator_layers.items():
        layers[f"cropland_{k}"] = cropland.multiply(image)
        layers[f"nonnat_{k}"] = nonnat.multiply(image)

    return layers

In [9]:
def _cache_layers(layers, scale, reduce_resolution=False, original_scale=30):
    """"""
    for k, image in layers.items():
        asset_id = f"{WORKING_FOLDER}/{k}_{scale}m"
        image = image.unmask()
        if reduce_resolution:
            image = (
                image.setDefaultProjection(PROJECTION, None, scale)
                .reduceResolution("mean", True)
                .reproject(PROJECTION, None, scale)
            )
        layers[k] = eeUtil.findOrSaveImage(image, asset_id, region=WORLD_GEOM, dtype="float")
    return layers


def _check_kernel_layers_precomputed():
    """The overlays should be done at a higher resolution before computing the kernels.

    There's a maximum number of pixels in a kernel, so the input data needs to be downsampled
    first, but we don't want to downsample the input data before overlaying.

    So first we compute the overlays at the ANALYSIS_SCALE, save these, then downsample to the
    KERNEL_SCALE and compute the kernels and final indicators.
    """
    kernel_assets = [
        f"tree_loss_to{TARGET_YEAR}_{ANALYSIS_SCALE}m",
        f"deforest_{ANALYSIS_SCALE}m",
        f"deforest_carbon_{ANALYSIS_SCALE}m",
        f"natcrop_expansion_{ANALYSIS_SCALE}m",
        f"natcrop_reduction_{ANALYSIS_SCALE}m",
        f"nonnatural_excl_builtwater_{ANALYSIS_SCALE}m",
    ]
    cached_assets = list(eeUtil.ls(f"{WORKING_FOLDER}"))
    return not any(k not in cached_assets for k in kernel_assets)


def compute_all_landscape_indicators(cache=True, export=True):
    """"""

    layers = get_derived_layers()

    precomputed = False
    if cache:
        layers = _cache_layers(layers, ANALYSIS_SCALE, reduce_resolution=True)
        precomputed = _check_kernel_layers_precomputed()
        if not precomputed:
            print("WARNING: Cannot compute final layers until overlay layers are computed.")
            print("Wait for current tasks to complete before re-running this cell.")
            print("Current tasks:", [t["description"] for t in eeUtil.getTasks(True)])
            return layers

    kernel_layers = compute_kernels(layers)
    if cache and precomputed:
        kernel_layers = _cache_layers(
            {f"{k}_{KERNEL_RADIUS/1000}km": i for k, i in kernel_layers.items()}, KERNEL_SCALE
        )

    indicators = compute_indicators(kernel_layers)
    if cache and precomputed:
        indicators = _cache_layers(
            {f"{k}_{KERNEL_RADIUS/1000}km": i for k, i in indicators.items()}, KERNEL_SCALE
        )

    viz_layers = compute_viz_layers(indicators)
    if cache and precomputed:
        viz_layers = _cache_layers(
            {f"{k}_{KERNEL_RADIUS/1000}km": i for k, i in viz_layers.items()}, KERNEL_SCALE
        )

    return {
        **layers,
        **kernel_layers,
        **indicators,
        **viz_layers,
    }

In [10]:
### Preview all layers


def vizualize(layers):
    map = gmap.Map()
    map.add_basemap("SATELLITE")
    for k, layer in layers.items():
        vmax = 1
        if "kernel" in k:
            vmax *= 0.2
        if "carbon" in k:
            vmax *= 50
        map.add_layer(
            layer.updateMask(layer.divide(vmax)),
            {"min": 0, "max": vmax * 2, "palette": ["black", "red", "orange", "yellow"]},
            k,
            False,
        )
    return map


vizualize(compute_all_landscape_indicators(cache=False))

In [11]:
# Compute and vizualize final layers
layers = compute_all_landscape_indicators(cache=True)

vizualize(layers)

Wait for current tasks to complete before re-running this cell.
Current tasks: ['projects:ee-fgassert:assets:landscape_indicators_20230811:natcrop_reduction_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:natcrop_expansion_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:cropland_reduction_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:cropland_expansion_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:deforest_carbon_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:deforest_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:cropland_2019_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:disturbance_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:gain_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:burned_area_mask_100m', 'projects:ee-fgassert:assets:landscape_indicators_20230811:modis_ba_100m', 'projects:ee-fgassert:assets:landscape

In [12]:
# make assets public
eeUtil.setAcl(WORKING_FOLDER, "public", recursive=True)